In [ ]:
### Import libraries
from netCDF4 import Dataset
import xarray as xr
import numpy as np
import cartopy.crs as crs
import matplotlib
from matplotlib.cm import get_cmap
import matplotlib.pyplot as plt
from wrf import getvar, interplevel, smooth2d
import warnings
from scipy.interpolate import griddata
from matplotlib.cm import get_cmap
import cartopy.crs as crs
from cartopy.feature import NaturalEarthFeature
warnings.filterwarnings('ignore')
import matplotlib.transforms as mtransforms
import matplotlib.ticker as mticker
from matplotlib.ticker import MultipleLocator

CP = 1005.7
RD = 287.04
P0 = 1000.
TR = 300.
LV = 2.501e6
EPS = 1.


In [ ]:
### Parameters
nx = 400
ny = 250
nz = 44
levs = np.arange(100,1010,10)

### Read in the data
ctl = Dataset('/p/work1/lloveras/nov2018/30km_files/ctl/wrfout_d01_2018-11-13_12_00_00')
adj = Dataset('/p/work1/lloveras/nov2018/30km_files/adj_full/wrfout_d01_2018-11-13_12_00_00')

### Pressure
p_ctl = np.asarray(getvar(ctl,'pressure'))
p_adj = np.asarray(getvar(adj,'pressure'))
p_avg = (p_ctl + p_adj)/2

### Theta
t_ctl = np.asarray(getvar(ctl,'THM')) + TR
t_adj = np.asarray(getvar(adj,'THM')) + TR
tk_ctl = t_ctl/((P0/p_ctl)**(RD/CP))
tk_adj = t_adj/((P0/p_adj)**(RD/CP))
dt = tk_adj - tk_ctl

### U
u_ctl = np.asarray(getvar(ctl,'ua'))
u_adj = np.asarray(getvar(adj,'ua'))
du = u_adj - u_ctl

### V
v_ctl = np.asarray(getvar(ctl,'va'))
v_adj = np.asarray(getvar(adj,'va'))
dv = v_adj - v_ctl

### DTE
dte = du**2 + dv**2 + (CP/TR)*dt**2
dte_p = np.asarray(interplevel(dte, p_avg, levs))
dte_prof = np.nanmean(dte_p,axis=(1,2))

In [ ]:
np.amax(np.sqrt(du**2 + dv**2))

In [ ]:
fig, axd = plt.subplot_mosaic([['left']],
                              constrained_layout=True, figsize=(6,5),dpi=200)


axd['left'].plot(dte_prof,levs,color='blue',linewidth=3.2)
axd['left'].grid()
axd['left'].set_xlabel('DTE (m$^2$ s$^{-2}$)',fontsize=18,labelpad=6)
axd['left'].yaxis.set_major_locator(MultipleLocator(100))
axd['left'].yaxis.set_minor_locator(MultipleLocator(50))
axd['left'].set_ylim([1050,50])
axd['left'].set_ylabel('Pressure (hPa)',fontsize=18,labelpad=6)
axd['left'].tick_params(axis='both', labelsize=18, which='major', pad=8,length=8)
axd['left'].tick_params(axis='both', which='minor', length=4)

plt.savefig('/p/work1/lloveras/nov2018/figs_png/dte_perts.png',bbox_inches='tight')